# EJFAT LB Control Plane Tester

This notebook stands up a slice of 3 nodes - sender, receiver and cpnode. The control plane daemon is deployed on the `cpnode` node in a 'mock' configuration (with no FPGA). DAQ and worker node code can be deployed on `sender` and `receiver` nodes for testing. The slice uses 'shared' and is created within a single FABRIC site for simplicity. It uses a single L2 bridge connection with RFC1918 IPv4 addressing, allowing all nodes to talk to each other. It is possible to run the dataplane assuming a single worker can keep up with a single sender since no actual load balancer is present in this configuration.

Slice example:

<div>
    <img src="figs/UDP LB Control Plane Testing slice.png" width=500>
</div>

## Preparation and overview

- Be sure to [generate a keypair for Jupyter Hub](GitHubSSH.ipynb) and register it with GitHub - the keys will be used to check out the code from private repositories, like [UDPLBd](https://github.com/esnet/udplbd) and [E2SAR](https://github.com/JeffersonLab/E2SAR).
- Note that for E2SAR development and testing sender and receiver node compile/build environments will be setup via post-boot scripts ([sender](post-boot/sender.sh) and [receiver](post-boot/recver.sh)) and grpc++ is installed as a tar.gz with static and dynamic libraries compiled for ubuntu22
- This does not setup the control plane node for anything, but testing a specific version - you can set which branch of UDPLBd to check out and a containerized version is built and stood up.

## Preamble

This cell must be executed whether you are creating a new slice or continuing work on the old one. If you are continuing work, you then skip the slice create section and proceed to wherever you left off.

In [ ]:
#
# EDIT THIS
#
# if you want to force a site instead of using random
# Pick 'UCSD', 'SRI', 'FIU' or 'TOKY' - these sites have
# IPv4. Other sites use IPv6 management and have trouble
# retrieving git-lfs artifacts.
site_override = 'SRI'
#site_override = None

# GitHub SSH key file (private) registered using the GitHubSSH.ipynb notebook referenced above
github_key = '/home/fabric/work/fabric_config/github_ecdsa'

# branches for UDPLBd and E2SAR that we want checked out on the VMs
udplbd_branch = 'develop'
e2sar_branch = 'main'

# which of the available config files to use with UDPLBd
udplbd_config = 'lb_mock-tls.yml'

# base distro 'ubuntu' or 'rocky'
distro_name = 'ubuntu'

binary_extension = {
    'ubuntu': 'ubuntu22',
    'rocky': 'rocky8'
}[distro_name]

# note that the below is distribution specific ('ubuntu' for ubuntu and so on)
home_location = {
    'ubuntu': '/home/ubuntu',
    'rocky' : '/home/rocky'
}[distro_name]

vm_key_location = f'{home_location}/.ssh/github_ecdsa'

# grpc tar ball stored in E2SAR repo (via LFS)
grpc_tar = f"grpc-v1.54.1-{binary_extension}.tar.gz"
boost_tar = f"boost-v1.85.0-{binary_extension}.tar.gz"

# which test suites in E2SAR to run (leave empty to run all)
# you can set 'unit' or 'live' to run unit or live tests only
e2sar_test_suite = ''

# name of the network connecting the nodes
net_name = 'site_bridge_net'


#
# SHOULDN'T NEED TO EDIT BELOW
#
# Preamble
from datetime import datetime
from datetime import timezone
from datetime import timedelta

from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

from ipaddress import ip_address, IPv4Address, IPv6Address, IPv4Network, IPv6Network
import ipaddress

fablib = fablib_manager()             
fablib.show_config();

images = {
    'ubuntu': ('default_ubuntu_22', 'docker_ubuntu_22'),
    'rocky': ('default_rocky_8', 'docker_rocky_8')
}

# variable settings
slice_name = f'UDP LB Control Plane Testing with udplbd[{udplbd_branch}], e2sar[{e2sar_branch}] on {distro_name}'

# for each node specify IP address (assuming /24), OS image
# note that most of the keys in these dictionaries map directly
# onto parameters to add_node()
node_config = {
    'sender': {
        'ip':'192.168.0.1', 
        'image': images[distro_name][0],
        'cores': 8,
        'ram': 24,
        'disk': 100 },
    'recver': {
        'ip':'192.168.0.2', 
        'image':images[distro_name][0],
        'cores':8,
        'ram': 24,
        'disk': 100 },
    'cpnode': {
        'ip':'192.168.0.3', 
        'image':images[distro_name][1],
        'cores':8,
        'ram': 8,
        'disk': 100 },
}
# skip these keys as they are not part of add_node params
skip_keys = ['ip']
# this is the NIC to use
nic_model = 'NIC_Basic'
# the subnet should match IPs
subnet = IPv4Network("192.168.1.0/24")

def execute_single_node(node, commands):
    for command in commands:
        print(f'\tExecuting "{command}" on node {node.get_name()}')
        #stdout, stderr = node.execute(command, quiet=True, output_file=node.get_name() + '_install.log')
        stdout, stderr = node.execute(command)
    if not stderr and len(stderr) > 0:
        print(f'Error encountered with "{command}": {stderr}')
        
def execute_commands(node, commands):
    if isinstance(node, list):
        for n in node:
            execute_single_node(n, commands)
    else:
        execute_single_node(node, commands)


## Create the slice

In [ ]:
# list all slices I have running
output_dataframe = fablib.list_slices(output='pandas')
if output_dataframe:
    print(output_dataframe)
else:
    print('No active slices under this project')

If your slice is already active you can skip to the 'Get Slice Details' section.

In [ ]:
# List available images (this step is optional)
available_images = fablib.get_image_names()

print(f'Available images are: {available_images}')

In [ ]:
# find an available site in continental US
lon_west=-124.3993243
lon_east=-69.9721573

# getting a random site make take a bit of time
if not site_override:
    selected_site = fablib.get_random_site(filter_function=lambda x: x['location'][1] < lon_east
                                              and x['location'][1] > lon_west) 
else:
    selected_site = site_override

if selected_site:
    print(f'Selected site is {selected_site}')
else:
    print('Unable to find a site matching the requirements')

# write selected site into node attributes
for n in node_config:
    node_config[n]['site'] = selected_site
    

In [ ]:
# build a slice
slice = fablib.new_slice(name=slice_name)

# create a network
net1 = slice.add_l2network(name=net_name, subnet=subnet)

nodes = dict()
# create  nodes for sending and receiving with a selected network card
# use subnet address assignment
for node_name, node_attribs in node_config.items():
    print(f"{node_name=} {node_attribs['ip']}")
    nodes[node_name] = slice.add_node(name=node_name, **{x: node_attribs[x] for x in node_attribs if x not in skip_keys})
    nic_interface = nodes[node_name].add_component(model=nic_model, name='_'.join([node_name, nic_model, 'nic'])).get_interfaces()[0]
    net1.add_interface(nic_interface)
    nic_interface.set_mode('config')
    nic_interface.set_ip_addr(node_attribs['ip'])
    # postboot configuration is under 'post-boot' directory
    nodes[node_name].add_post_boot_upload_directory('post-boot','.')
    nodes[node_name].add_post_boot_execute(f'chmod +x post-boot/{node_name}.sh && ./post-boot/{node_name}.sh')

print(f'Creating a {distro_name} based slice named "{slice_name}" with nodes in {selected_site}')

# Submit the slice
slice.submit();

## Get Slice Details

If not creating a new slice, and just continuing work on an existing one, execute this cell (in addition to the preamble) and then any of the cells below will work.

In [ ]:
# get slice details (if not creating new)
slice = fablib.get_slice(name=slice_name)
a = slice.show()
nets = slice.list_networks()
nodes = slice.list_nodes()

cpnode = slice.get_node(name="cpnode")    
sender = slice.get_node(name="sender")
recver = slice.get_node(name="recver")


# get node dataplane addresses
cpnode_addr = cpnode.get_interface(network_name=net_name).get_ip_addr()
sender_addr = sender.get_interface(network_name=net_name).get_ip_addr()
recver_addr = recver.get_interface(network_name=net_name).get_ip_addr()

## Start the UDPLBd container

In [ ]:
# check if any dockers are running already and that we have compose and buildx installed by post-boot script
commands = [
    'docker container ls',
    'docker compose version',
    'docker buildx version'
]
execute_commands(cpnode, commands)

In [ ]:
# upload the mock config file for UDPLBd 
result = cpnode.upload_file(f'config/{udplbd_config}','lb_mock.yml')

# upload the GitHub SSH key onto the VM
result = cpnode.upload_file(github_key, vm_key_location)

# checkout UDPLBd (including the right branch) using that key
commands = [
    f"chmod go-rwx {vm_key_location}",
    f"GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone -b {udplbd_branch} git@github.com:esnet/udplbd.git",
]

execute_commands(cpnode, commands)

In [ ]:
# copy configuration file into place, generate self-signed cert and start the UDPLBd container
commands = [
    f'cp lb_mock.yml ./udplbd/etc/config.yml',
    f'openssl req -x509 -newkey rsa:4096 -keyout udplbd/etc/server_key.pem -out udplbd/etc/server_cert.pem -sha256 -days 365 -nodes -subj "/CN=cpnode/subjectAltName=IP:{cpnode_addr}" -nodes',
    f'cd udplbd; docker compose up -d'
]

execute_commands(cpnode, commands)

In [ ]:
# check the logs
commands = [
    'docker compose ls',
    'cd udplbd; docker compose logs'
]

execute_commands(cpnode, commands)

In [ ]:
# if you need to restart it, this is the stop part
commands = [
    'cd udplbd; docker compose stop; docker compose rm -f; docker image rm udplbd'
]

execute_commands(cpnode, commands)

## Clone E2SAR code into sender and receiver

In [ ]:
# install github ssh key and set up build environment variables for interactive logins
commands = [
    f"chmod go-rwx {vm_key_location}",
    # Meson won't detect boost by merely setting cmake_prefix_path, instead set BOOST_ROOT env variable 
    # for gRPC it is enough to set -Dpkg_config_path option to meson
    f"echo 'export BOOST_ROOT=$HOME/boost-install PATH=$HOME/.local/bin:$HOME/grpc-install/bin:$PATH LD_LIBRARY_PATH=$HOME/grpc-install/lib/:$HOME/boost-install/lib' >> ~/.profile",
    f"echo 'export BOOST_ROOT=$HOME/boost-install PATH=$HOME/.local/bin:$HOME/grpc-install/bin:$PATH LD_LIBRARY_PATH=$HOME/grpc-install/lib/:$HOME/boost-install/lib' >> ~/.bashrc",
]

for node in [sender, recver]:    
    # upload the GitHub SSH key onto the VM
    result = node.upload_file(github_key, vm_key_location)
    execute_commands(node, commands)

In [ ]:
# checkout E2SAR (including the right branch) using that key, install grpc and boost binary that is stored in the repo
commands = [
    f"GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git clone --recurse-submodules --depth 1 -b {e2sar_branch} git@github.com:JeffersonLab/E2SAR.git",
    f"tar -zxf E2SAR/binary_artifacts/{grpc_tar}",
    f"tar -zxf E2SAR/binary_artifacts/{boost_tar}",
    #f"cd E2SAR; GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git submodule init",
]
 
execute_commands([sender, recver], commands)

On RedHat/Rocky derivatives the build fails. Likely has to do with the outdated gcc/g++ and stdlib. E2SAR/build/ninja.build must be modified - all mentions of `-std=c++11` must be removed from it.

To avoid this after the build step (and the next one after) the below command invokes `sed` to remove mentions of `c++11` from build.ninja file. This is a hack, don't just walk away, **RUN**!

Also note that the install directory is set to `$(HOME)/e2sar-install`. So if you run `meson install -C build` this is where the code will end up.

In [ ]:
# build E2SAR code and run unit tests
commands = [
    f"cd E2SAR; BOOST_ROOT=$HOME/boost-install PATH=$HOME/.local/bin:$HOME/grpc-install/bin:$PATH LD_LIBRARY_PATH=$HOME/grpc-install/lib/:$HOME/boost-install/lib meson setup -Dpkg_config_path=$HOME/grpc-install/lib/pkgconfig/:$HOME/grpc-install/lib64/pkgconfig/ --prefix {home_location}/e2sar-install build && sed -i 's/-std=c++11//g' build/build.ninja",
    f"cd E2SAR/build; PATH=$HOME/.local/bin:$HOME/grpc-install/bin:$PATH LD_LIBRARY_PATH=$HOME/grpc-install/lib/:$HOME/boost-install/lib  meson compile -j 8",
    f"cd E2SAR/build; EJFAT_URI='ejfats://udplbd@{cpnode_addr}:18347/lb/12?sync=192.168.100.10:19020&data=192.168.101.10:18020' PATH=$HOME/.local/bin:$HOME/grpc-install/bin:$PATH LD_LIBRARY_PATH=$HOME/grpc-install/lib/:$HOME/boost-install/lib  meson test {e2sar_test_suite} --suite unit  --timeout 0",
    f"cd E2SAR/build; EJFAT_URI='ejfats://udplbd@{cpnode_addr}:18347/' PATH=$HOME/.local/bin:$HOME/grpc-install/bin:$PATH LD_LIBRARY_PATH=$HOME/grpc-install/lib/:$HOME/boost-install/lib  meson test {e2sar_test_suite} --suite live --timeout 0"
]
 
execute_commands([sender, recver], commands)

Again, making sure we update ninja.build... Look away...

In [ ]:
# update the code, compile and test

commands = [
    f"cd E2SAR; GIT_SSH_COMMAND='ssh -i {vm_key_location} -o IdentitiesOnly=yes -o UserKnownHostsFile=/dev/null -o StrictHostKeyChecking=no' git pull origin {e2sar_branch}",
    f"cd E2SAR; BOOST_ROOT=$HOME/boost-install PATH=$HOME/.local/bin:$HOME/grpc-install/bin:$PATH LD_LIBRARY_PATH=$HOME/grpc-install/lib/:$HOME/boost-install/lib meson setup -Dpkg_config_path=$HOME/grpc-install/lib/pkgconfig/:$HOME/grpc-install/lib64/pkgconfig/ --prefix {home_location}/e2sar-install build --wipe && sed -i 's/-std=c++11//g' build/build.ninja",
    f"cd E2SAR/build; PATH=$HOME/.local/bin:/home/ubuntu/grpc-install/bin:$PATH LD_LIBRARY_PATH=$HOME/grpc-install/lib/:$HOME/boost-install/lib  meson compile -j 8",
    f"cd E2SAR/build; EJFAT_URI='ejfats://udplbd@{cpnode_addr}:18347/lb/12?sync=192.168.100.10:19020&data=192.168.101.10:18020' PATH=$HOME/.local/bin:$HOME/grpc-install/bin:$PATH LD_LIBRARY_PATH=$HOME/grpc-install/lib/:$HOME/boost-install/lib  meson test {e2sar_test_suite} --suite unit --timeout 0",
    f"cd E2SAR/build; EJFAT_URI='ejfats://udplbd@{cpnode_addr}:18347/' PATH=$HOME/.local/bin:$HOME/grpc-install/bin:$PATH LD_LIBRARY_PATH=$HOME/grpc-install/lib/:$HOME/boost-install/lib  meson test {e2sar_test_suite} --suite live --timeout 0"
]
  
execute_commands([sender, recver], commands)

## Manage the slice

### Extend

In [ ]:
# Set end host to now plus 14 days
end_date = (datetime.now(timezone.utc) + timedelta(days=14)).strftime("%Y-%m-%d %H:%M:%S %z")

try:
    slice = fablib.get_slice(name=slice_name)

    slice.renew(end_date)
except Exception as e:
    print(f"Exception: {e}")

### Delete

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()